# Estimate wind speeds from flight data



In [1]:
from flightanalysis import Section
import plotly.express as px
from geometry import Points
from geometry import Point


secs = [
    Section.from_csv("examples/data/P23.csv").flying_only(),
    Section.from_csv("examples/data/loops.csv").flying_only(),
    Section.from_csv("examples/data/snap_roll_section.csv").flying_only(),
]


In [2]:
from flightplotting.plots import plotsec

plotsec(secs[0])

Difference between airspeed * body x axis and the velocity vector gives the wind vector. But, airspeed is not known. 

proposed solution - assume a wind field, direction and magnitude. Subtract the local wind speed from the body velocity vector. Resulting vector should have the same heading as the body x axis. use the direction and magnitude parameters as variables to minimize the heading error between the two vectors. 



In [3]:
import numpy as np
def wind_speed(h, v0, h0=300.0, a=0.2):
    return v0 * (h/h0)**a

def wind_vector(head, h, v0, h0=300.0, a=0.2):
    speed = wind_speed(h, v0, h0, a)
    try:
        return np.array([speed * np.cos(head), speed * np.sin(head), np.zeros(len(h))])
    except TypeError:
        return np.array([speed * np.cos(head), speed * np.sin(head), 0])

h = np.linspace(0, 300, 1000)
vecs = Points(wind_vector(np.radians(30), h, 20).T)
vecdf = vecs.to_pandas()
vecdf.index = h
px.line(vecdf, width=500, height=300)

In [4]:

def get_wind_error(args: np.ndarray, sec: Section) -> float:
    #wind vectors at the local positions
    local_wind = Points(wind_vector(head=args[0], h=np.maximum(sec.gpos.z, 0), v0=args[1], a=args[2]).T)

    #wind vectors in body frame
    body_wind = sec.gatt.inverse().transform_point(local_wind)

    #body frame velocity - wind vector should be wind axis velocity
    #error in wind axis velocity, is the non x axis part (because we fly forwards)
    air_vec_error = (sec.gbvel - body_wind) * Point(0,1,1)

    #but the wind is only horizontal, so transform to world frame and remove the z component
    world_air_vec_error = sec.gatt.transform_point(air_vec_error) * Point(1,1,0)

    #the cost is the cumulative error
    return sum(abs(world_air_vec_error))

In [18]:
from scipy.optimize import minimize, Bounds
winds = [minimize(get_wind_error, [np.pi, 5.0, 0.2], args=sec, method='Nelder-Mead') for sec in secs]
for wind in winds:
    if wind.x[1] < 0:
        wind.x[1] = -wind.x[1]
        wind.x[0] = (wind.x[0] + np.pi) % (2 * np.pi) 
print([wind.x for wind in winds])

def local_wind(position, secid):
    return wind_vector(head=winds[secid].x[0], h=np.maximum(position.z, 0), v0=winds[secid].x[1], a=winds[secid].x[2])
    
#[array([6.15248161, 2.68312782, 0.48702976]), array([ 5.98325395, 11.73178835,  0.57953661]), array([6.15248161, 2.68312782, 0.48702976])]

[array([6.16549326, 2.37530382, 0.18969299]), array([5.99963945, 9.5050707 , 0.19052333]), array([6.16549326, 2.37530382, 0.18969299])]


In [6]:
import numpy as np
import pandas as pd

for i, wind in enumerate(winds):
    heads = np.linspace(0.0, 2 * np.pi, 50)
    mags = np.linspace(max(0.0, wind.x[1] - 3.0), wind.x[1] + 3.0, 10)

    errs = [(mag, head, get_wind_error([head, mag, wind.x[2]], secs[i])) for head in heads for mag in mags]
    df = pd.DataFrame(errs, columns=["windspeed", "heading", "cost"])
    px.scatter(df, x="heading", y="cost", color="windspeed", title="Section {}".format(i)).show()

In [15]:

secid = 1
test = secs[secid].subset(420.3, 429.4)
#plotsec(test, nmodels=2).show()

test_states = [
    ["calm_upline", 0, 109.4],
    ["calm_downline", 0, 125.4],
    ["inverted", 0, 129],
    ["upright", 0, 172.9],
    ["windy_upline", 1, 420.3],
    ["windy_downline", 1, 420.3],
]

names = [state[0] for state in test_states]
states = [secs[tstate[1]].get_state_from_time(tstate[2]) for tstate in test_states]
wind_speeds = [Point(*local_wind(state.pos, tstate[1])) for state, tstate in zip(states, test_states)]

#print(names, states, wind_speeds)
#df =pd.DataFrame(np.array([names, states, winds]).T, columns=["names", "states", "winds"])

In [11]:
from flightplotting.plots import plotsec
from geometry import Quaternion
from flightanalysis import State




def aerodynamic_parameters(state: State, wind_vec: Point):
    print("Atmospheric Wind Velocity in Sequence Frame = {}".format(wind_vec))

    body_wind_vec = state.att.inverse().transform_point(wind_vec) 
    print("Atmospheric Wind Velocity in Body Frame = {}".format(body_wind_vec))

    print("Velocity in Body Frame = {}".format(state.bvel))

    print("Velocity in World Frame = {}".format(state.att.transform_point(state.bvel)))

    body_oncoming_wind=state.bvel - body_wind_vec
    print("Body Wind Vector = {}".format(body_oncoming_wind))

    alpha = np.tan(body_oncoming_wind.z / body_oncoming_wind.x)
    beta = np.tan(body_oncoming_wind.y / body_oncoming_wind.x)
    print("Angle of Attack, Alpha = {}, Beta = {}".format(np.degrees(alpha), np.degrees(beta)) )

    print("Body Frame Accelerations (m/s**2) = {}".format(state.bacc *9.81 ))

    forces = 4.5 * state.bacc # 
    print("Body Frame Forces (kgf) = {}".format(forces / 9.81 ))

    #print("At this point need to s")

    body_to_stability = Quaternion.from_euler((0.0, alpha, 0.0))
    stability_to_wind = Quaternion.from_euler((0.0, 0.0, -beta))

    wind_force = stability_to_wind.transform_point(body_to_stability.transform_point(forces))

    print("Wind Axis Lift Force (kgf) = {}".format(wind_force / 9.81 ))



for name, state, wind in zip(names, states, wind_speeds):
    print(name)
    aerodynamic_parameters(state, wind)
    print("\n")

#plotsec(test, nmodels=5 )

calm_upline
Atmospheric Wind Velocity in Sequence Frame = X:1.81,Y:-0.24,Z:0.00
Atmospheric Wind Velocity in Body Frame = X:-0.15,Y:-0.29,Z:-1.79
Velocity in Body Frame = X:25.47,Y:-0.10,Z:-2.96
Velocity in World Frame = X:0.97,Y:0.97,Z:25.61
Body Wind Vector = X:25.62,Y:0.19,Z:-1.17
Angle of Attack, Alpha = -2.607541201309227, Beta = 0.4340706516528073
Body Frame Accelerations (m/s**2) = X:85.70,Y:-6.89,Z:10.68
Body Frame Forces (kgf) = X:4.01,Y:-0.32,Z:0.50
Wind Axis Lift Force (kgf) = X:3.98,Y:-0.35,Z:0.68


calm_downline
Atmospheric Wind Velocity in Sequence Frame = X:1.83,Y:-0.24,Z:0.00
Atmospheric Wind Velocity in Body Frame = X:-0.05,Y:0.58,Z:-1.75
Velocity in Body Frame = X:38.81,Y:0.90,Z:-1.84
Velocity in World Frame = X:1.25,Y:1.87,Z:-38.80
Body Wind Vector = X:38.86,Y:0.32,Z:-0.09
Angle of Attack, Alpha = -0.1271306323257202, Beta = 0.4702049602249742
Body Frame Accelerations (m/s**2) = X:-89.75,Y:-1.73,Z:5.70
Body Frame Forces (kgf) = X:-4.20,Y:-0.08,Z:0.27
Wind Axis Lift F

In [ ]:
# alpha, beta, axis_rates, airspeed, control_deflections  - > forces + moments

In [ ]:
from flightplotting.plots import aoa_brv_plot
from flightplotting.traces import sec_col_trace
import plotly.graph_objects as go
from plotly.subplots import mQQake_subplots
import numpy as np
snap = schedule.manoeuvres[16].elements[2].get_data(aligned)
aoa_brv_plot(snap)

In [ ]:
import numpy as np
import pandas as pd

pd.DataFrame([]).reindex()